### Managing Nulls with Pandas

In this notebook, we will take a look at some ways to manage nulls using Pandas DataFrames.

For even more details on how to do this, check out the [Panda's documentation](http://pandas.pydata.org/pandas-docs/stable/missing_data.html).

### Run the below code cell to download the dataset

In [1]:
!wget https://cdn.talentsprint.com/aiml/iot_example_with_nulls.csv

--2022-09-04 06:05:31--  https://cdn.talentsprint.com/aiml/iot_example_with_nulls.csv
Resolving cdn.talentsprint.com (cdn.talentsprint.com)... 172.105.34.236
Connecting to cdn.talentsprint.com (cdn.talentsprint.com)|172.105.34.236|:443... connected.
HTTP request sent, awaiting response... 302 Moved Temporarily
Location: https://cdn.ap-south-1.linodeobjects.com/public_html/aiml/iot_example_with_nulls.csv [following]
--2022-09-04 06:05:32--  https://cdn.ap-south-1.linodeobjects.com/public_html/aiml/iot_example_with_nulls.csv
Resolving cdn.ap-south-1.linodeobjects.com (cdn.ap-south-1.linodeobjects.com)... 172.104.56.205, 103.3.61.236, 172.104.36.102, ...
Connecting to cdn.ap-south-1.linodeobjects.com (cdn.ap-south-1.linodeobjects.com)|172.104.56.205|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 10972734 (10M) [text/csv]
Saving to: ‘iot_example_with_nulls.csv’

iot_example_with_nu 100%[===================>]  10.46M  5.39MB/s    in 1.9s    

2022-09-04 06:05:34 (

In [2]:
import pandas as pd
from numpy import random

In [3]:
df = pd.read_csv('iot_example_with_nulls.csv')

### Data Quality Check

In [4]:
df.head()

,timestamp,username,temperature,heartrate,build,latest,note
0,2017-01-01T12:00:23,michaelsmith,12.0,67,4e6a7805-8faa-2768-6ef6-eb3198b483ac,0.0,interval
1,2017-01-01T12:01:09,kharrison,6.0,78,7256b7b0-e502-f576-62ec-ed73533c9c84,0.0,wake
2,2017-01-01T12:01:34,smithadam,5.0,89,9226c94b-bb4b-a6c8-8e02-cb42b53e9c90,0.0,NaN
3,2017-01-01T12:02:09,eddierodriguez,28.0,76,NaN,0.0,update
4,2017-01-01T12:02:36,kenneth94,29.0,62,122f1c6a-403c-2221-6ed1-b5caa08f11e0,NaN,NaN


In [5]:
df.dtypes

timestamp       object
username        object
temperature    float64
heartrate        int64
build           object
latest         float64
note            object
dtype: object

In [6]:
df.note.value_counts()

wake        16496
user        16416
interval    16274
sleep       16226
update      16213
test        16068
Name: note, dtype: int64

### Let's remove all null values (including the note: n/a)

In [8]:
df = pd.read_csv('iot_example_with_nulls.csv', 
                 na_values=['n/a'])

### Test to see if we can use dropna

In [9]:
df.shape

(146397, 7)

In [10]:
df.dropna().shape

(46116, 7)

In [34]:
df.dropna(how='all', axis=1).shape

(146397, 6)

### Test to see if we can drop columns

In [12]:
my_columns = list(df.columns)

In [13]:
my_columns

['timestamp',
 'username',
 'temperature',
 'heartrate',
 'build',
 'latest',
 'note']

In [14]:
list(df.dropna(thresh=int(df.shape[0] * .9), axis=1).columns)

['timestamp', 'username', 'heartrate']

### I want to find all columns that have missing data

In [15]:
missing_info = list(df.columns[df.isnull().any()])

In [16]:
missing_info

['temperature', 'build', 'latest', 'note']

In [17]:
for col in missing_info:
    num_missing = df[df[col].isnull() == True].shape[0]
    print('number missing for column {}: {}'.format(col, 
                                                    num_missing))

number missing for column temperature: 32357
number missing for column build: 32350
number missing for column latest: 32298
number missing for column note: 48704


In [18]:
for col in missing_info:
    percent_missing = df[df[col].isnull() == True].shape[0] / df.shape[0]
    print('percent missing for column {}: {}'.format(
        col, percent_missing))

percent missing for column temperature: 0.22102228870810195
percent missing for column build: 0.22097447352063226
percent missing for column latest: 0.22061927498514314
percent missing for column note: 0.332684412931959


### Can I easily substitute majority values in for missing data?

In [19]:
df.note.value_counts()

wake        16496
user        16416
interval    16274
sleep       16226
update      16213
test        16068
Name: note, dtype: int64

In [20]:
df.build.value_counts().head()

4e6a7805-8faa-2768-6ef6-eb3198b483ac    1
12aefc6b-272c-751e-6117-134ee73e2649    1
fd4049c3-2297-14ac-a27e-6da57129dd10    1
0bcfab8f-bc25-3f8f-8585-0614e1555fd1    1
b0de05dd-2860-abbb-8be6-f5c0e30ca063    1
Name: build, dtype: int64

In [21]:
df.latest.value_counts()

0.0    75735
1.0    38364
Name: latest, dtype: int64

In [25]:
df.latest = df.latest.fillna(0)
df.head()

,timestamp,username,temperature,heartrate,build,latest,note
0,2017-01-01T12:00:23,michaelsmith,12.0,67,4e6a7805-8faa-2768-6ef6-eb3198b483ac,0.0,interval
1,2017-01-01T12:01:09,kharrison,6.0,78,7256b7b0-e502-f576-62ec-ed73533c9c84,0.0,wake
2,2017-01-01T12:01:34,smithadam,5.0,89,9226c94b-bb4b-a6c8-8e02-cb42b53e9c90,0.0,NaN
3,2017-01-01T12:02:09,eddierodriguez,28.0,76,NaN,0.0,update
4,2017-01-01T12:02:36,kenneth94,29.0,62,122f1c6a-403c-2221-6ed1-b5caa08f11e0,0.0,NaN


### Have not yet addressed temperature missing values... Let's find a way to fill

In [26]:
df.username.value_counts().head()

esmith    45
zsmith    43
vsmith    41
ysmith    40
jsmith    37
Name: username, dtype: int64

In [30]:
df = df.set_index('timestamp')

In [31]:
df.head()

,username,temperature,heartrate,build,latest,note
timestamp,,,,,,
2017-01-01T12:00:23,michaelsmith,12.0,67,4e6a7805-8faa-2768-6ef6-eb3198b483ac,0.0,interval
2017-01-01T12:01:09,kharrison,6.0,78,7256b7b0-e502-f576-62ec-ed73533c9c84,0.0,wake
2017-01-01T12:01:34,smithadam,5.0,89,9226c94b-bb4b-a6c8-8e02-cb42b53e9c90,0.0,NaN
2017-01-01T12:02:09,eddierodriguez,28.0,76,NaN,0.0,update
2017-01-01T12:02:36,kenneth94,29.0,62,122f1c6a-403c-2221-6ed1-b5caa08f11e0,0.0,NaN


In [36]:
df.temperature = df.groupby('username').temperature.fillna(
    method='backfill', limit=3)

In [37]:
df

,username,temperature,heartrate,build,latest,note
timestamp,,,,,,
2017-01-01T12:00:23,michaelsmith,12.0,67,4e6a7805-8faa-2768-6ef6-eb3198b483ac,0.0,interval
2017-01-01T12:01:09,kharrison,6.0,78,7256b7b0-e502-f576-62ec-ed73533c9c84,0.0,wake
2017-01-01T12:01:34,smithadam,5.0,89,9226c94b-bb4b-a6c8-8e02-cb42b53e9c90,0.0,NaN
2017-01-01T12:02:09,eddierodriguez,28.0,76,NaN,0.0,update
2017-01-01T12:02:36,kenneth94,29.0,62,122f1c6a-403c-2221-6ed1-b5caa08f11e0,0.0,NaN
...,...,...,...,...,...,...
2017-02-28T23:58:06,mcontreras,15.0,63,69e61a15-d2d0-47a7-1a27-e07b3eeeba10,0.0,NaN
2017-02-28T23:58:43,joelrusso,NaN,89,NaN,0.0,NaN
2017-02-28T23:59:23,lellis,NaN,84,dac87426-e147-9c39-6e4c-790bb11f8fc9,0.0,update
